In [ ]:
from statistic_helper import *
from graph_helper import *
import plotly.offline as po
from plotly.graph_objs import *
import plotly.graph_objs as go
import json, nltk
import numpy as np
po.offline.init_notebook_mode()

In [ ]:
students = load_data_from_file("studentBehaviorInfo_all.json")
video_data_processed = load_data_from_file("video_data.json")
video_raw = load_data_from_file("videoData_vs_WordList.json")

In [ ]:
# TODO:
# Explore info about saved words and score (Some student saved tons of words in a video and get 0 score), ex: What's special
# in the videos that user saved most of the words? Does that mean that it's difficult for them?
# Try data cleaning based on score and saved words <- my main goal

In [ ]:
#Overall user chosen video difficulty and score
cleaning = True
i=0
data = []
student_summary = []
for student in students:
    totalScore = 0
    totalSession = 0
    stats = []
    ignoring = False
    for video in student['chosenVideo']:
        videoSpeed = video_data_processed[str(video)]['videoSpeed']
        wordLevel = video_data_processed[str(video)]['wordLevel']
        total = 0
        amount = 0
        skipped_tests = 0
        sessions = 0
        scores = []
        vocabularyList = []
        ignoring2 = False
        for wd in student['vocabularyList']:
            if int(wd["postId"]) == video:
                vocabularyList.append(wd['word'])
        for score in student['listenScore']:
            if score['postId'] == video:
                if score['score'] == -1:
                    skipped_tests += 1
                else:
                    scores.append(score)
                    amount += 1
                    total += score['score']
                sessions += 1
        if amount != 0:
            totalScore += total
            totalSession += amount
            average = float(total)/amount
            stats.append({"postId": video,"videoSpeed": videoSpeed,"wordLevel": wordLevel,"scores": scores,"average": average,"skipped_tests"
                          : skipped_tests,"sessions": sessions, 'vocabularyList': vocabularyList})
    
    student_summary.append({"memberId": student['memberId'], "average": float(totalScore)/totalSession, "totalSession": totalSession, "stats":
                            stats})
#         print (video, videoSpeed, wordLevel, average, skipped_tests)

    trace0 = go.Scatter(
    x = range(len(stats)),
    y = [stat["wordLevel"] for stat in stats])
    
    trace1 = go.Scatter(
    x = range(len(stats)),
    y = [stat["average"] for stat in stats])
    i += 1
    data.append(trace0)
    data.append(trace1)
    
# po.iplot(data)


In [ ]:
looking_for = '10019'
for vid in video_raw:
    if vid['postId'] == looking_for:
        print vid
        break
bk = False
for student in student_summary:
    for stat in student['stats']:
        if str(stat['postId']) == looking_for:
            print stat['sessions']
            print stat['wordLevel']
            bk = True
            break
        if bk:
            break

In [ ]:
# Comparison of student score and videos/lessons taken
vids_x = []
sessions_x = []
avg = []
for student in student_summary:
    vids_x.append(len(student['stats']))
    sessions_x.append(student['totalSession'])
    avg.append(student['average'])

#Regression

z = np.polyfit(vids_x, avg, 1)
f = np.poly1d(z)
print f

z2 = np.polyfit(sessions_x, avg, 1)
f2 = np.poly1d(z)
print f2
    
trace_vid = go.Scatter(
    x = vids_x,
    y = avg,
    mode = "markers")
data = [trace_vid]

po.iplot(data)


trace_session = go.Scatter(
    x = sessions_x,
    y = avg,
    mode = "markers")
data = [trace_session]

po.iplot(data)


In [ ]:
for student in student_summary:
    if 0 < student['totalSession'] < 100 and 0 < student['average'] < 20:
        stats = student["stats"]
        trace0 = go.Scatter(
        x = ["label: " + str(stat['scores'][0]['postId']) for stat in stats],
        y = [stat["wordLevel"] for stat in stats],
        name = 'Word Level')

        trace1 = go.Scatter(
        x = ["label: " + str(stat['scores'][0]['postId']) for stat in stats],
        y = [stat["average"] for stat in stats],
        name = "Average Score")

        trace2 = go.Scatter(
        x = ["label: " + str(stat['scores'][0]['postId']) for stat in stats],
        y = [len(stat['vocabularyList']) for stat in stats],
        name = 'Amount of saved words')
        data = [trace0, trace1, trace2]

        layout = go.Layout(
            title = str(student['memberId'])
        )

#         fig = go.Figure(data=data, layout=layout)
#         po.iplot(fig)

In [ ]:
# CSV export



import csv
with open('../data/user average score sequence.csv', 'wb') as out:
    writer = csv.writer(out)
    
    row = ['memberId']
    for label in range(1,151):
        row.append('video'+str(label))
    writer.writerow(row)
    
    for student in student_summary:
        row = [student['memberId']]
        for stat in student['stats']:
            row.append(stat['average'])
        row.append('end')
        writer.writerow(row)
        
with open('../data/user saved word sequence.csv', 'wb') as out:
    writer = csv.writer(out)
    
    row = ['memberId']
    for label in range(1,151):
        row.append('video'+str(label))
    writer.writerow(row)
    
    for student in student_summary:
        row = [student['memberId']]
        for stat in student['stats']:
            row.append(len(stat['vocabularyList']))
        row.append('end')
        writer.writerow(row)

In [ ]:
# Feature extraction from score/save word sequence
# zero rate

# rising rate
# falling rate
# flip rate
# length of sequence zero / all
# length of rising / all
# length of falling / all

student_features = []
for student in student_summary:
    zero_count = 0
    rising_count = 0
    falling_count = 0
    flip_count = 0
    video_count = 0
    max_sequence_zero = 0
    curr_sequence_zero = 0
    max_sequence_rising = 0
    curr_sequence_rising = 0
    max_sequence_falling = 0
    curr_sequence_falling = 0
    former_score = -2
    for stat in student['stats']:
        score = stat['average']
        if former_score != -2:
            if score == 0:
                zero_count += 1
                curr_sequence_zero += 1
                max_sequence_zero = max(max_sequence_zero, curr_sequence_zero)
            else:
                curr_sequence_zero = 0
            if score > former_score:
                if curr_sequence_falling > 0:
                    flip_count += 1
                    curr_sequence_falling = 0
                rising_count += 1
                curr_sequence_rising += 1
                max_sequence_rising = max(max_sequence_rising, curr_sequence_rising)
            elif score < former_score:
                if curr_sequence_rising > 0:
                    flip_count += 1
                    curr_sequence_rising = 0
                falling_count += 1
                curr_sequence_falling += 1
                max_sequence_falling = max(max_sequence_falling, curr_sequence_falling)
            else:
                if curr_sequence_falling > 0:
                    curr_sequence_falling += 1
                    max_sequence_falling = max(max_sequence_falling, curr_sequence_falling)
                if curr_sequence_rising:
                    curr_sequence_rising += 1
                    max_sequence_rising = max(max_sequence_rising, max_sequence_rising)
        elif score == 0:
            zero_count+= 1
            curr_sequence_zero += 1
            max_sequence_zero += 1
        former_score = score
        video_count += 1
    zero_rate = zero_count / float(video_count)
    if video_count > 1:
        rising_rate = rising_count / float(video_count-1)
        falling_rate = falling_count / float(video_count-1)
    else:
        rising_rate = 0
        falling_rate = 0
    if video_count > 2:
        flip_rate = flip_count / float(video_count-2)
    else:
        flip_rate = 0
    zero_sequence_rate = max_sequence_zero / float(video_count)
    if video_count > 1:
        rising_sequence_rate = max_sequence_rising / float(video_count-1)
        falling_sequence_rate = max_sequence_falling / float(video_count-1)
    else:
        rising_sequence_rate = 0
        falling_sequence_rate = 0
        
    student_features.append({"memberId": student["memberId"], "zero_rate": zero_rate, "rising_rate": rising_rate, 
                            "falling_rate": falling_rate, "flip_rate": flip_rate, "rising_sequence_rate": rising_sequence_rate,
                            "falling_sequence_rate": falling_sequence_rate, "zero_sequence_rate": zero_sequence_rate})
    
with open('../data/user_score_features.csv', 'wb') as out:
    writer = csv.writer(out)
    
    row = ['memberId', 'zero_rate', 'rising_rate', 'falling_rate', 'flip_rate', 'rising_sequence_rate', 'falling_sequence_rate', 'zero_sequence_rate']
    writer.writerow(row)
    
    for feature in student_features:
        row = [feature['memberId'], feature['zero_rate'], feature['rising_rate'], feature['falling_rate'], feature['flip_rate'], 
              feature['rising_sequence_rate'], feature['falling_sequence_rate'], feature['zero_sequence_rate']]
        writer.writerow(row)
        
zero_rate_data = [feature['zero_rate'] for feature in student_features]
rising_rate_data = [feature['rising_rate'] for feature in student_features]
falling_rate_data = [feature['falling_rate'] for feature in student_features]
flip_rate_data = [feature['flip_rate'] for feature in student_features]
zero_sequence_rate_data = [feature['zero_sequence_rate'] for feature in student_features]
rising_sequence_rate_data = [feature['rising_sequence_rate'] for feature in student_features]
falling_sequence_rate_data = [feature['falling_sequence_rate'] for feature in student_features]
data = [go.Histogram(
        x=zero_rate_data,
        name='Zero rate'),
        go.Histogram(
        x=rising_rate_data,
        name = 'Rising rate'),
        go.Histogram(
        x=falling_rate_data,
        name='Falling rate'),
        go.Histogram(
        x=flip_rate_data,
        name='Flipping rate'),
        go.Histogram(
        x=zero_sequence_rate_data,
        name='Length of zero sequence'),
        go.Histogram(
        x=rising_sequence_rate_data,
        name='Length of rising sequence'),
        go.Histogram(
        x=falling_sequence_rate_data,
        name='Length of falling sequence')
       ]
layout = go.Layout(
    title='Distribution of Features',
    barmode='group')

fig = go.Figure(data=data, layout=layout)
po.iplot(fig)

In [15]:
# CLustering
import scipy
from sklearn.cluster import KMeans
from sklearn import datasets

np.random.seed(420420)
data = []
for feature in student_features:
    data.append([feature['zero_rate'], feature['rising_rate'], feature['falling_rate'], feature['flip_rate'],
                 feature['rising_sequence_rate'], feature['falling_sequence_rate'], feature['zero_sequence_rate']])

est = KMeans(n_clusters=3)
est.fit(data)
labels = est.labels_

num=0
for feature in student_features:
    feature['label'] = labels[num]
    num+=1
    

for label_no in range(3):
    data = []
    for student in student_summary:
        for feature in student_features:
            if feature['memberId'] == student['memberId'] and feature['label'] == label_no:
                stats = student["stats"]
    #             trace0 = go.Scatter(
    #             x = ["label: " + str(stat['scores'][0]['postId']) for stat in stats],
    #             y = [stat["wordLevel"] for stat in stats],
    #             name = 'Word Level')

                trace1 = go.Scatter(
                x = [num for num in range(1, len(stats)+1)],
                y = [stat["average"] for stat in stats],
                name = str(student['memberId']))

    #             trace2 = go.Scatter(
    #             x = ["label: " + str(stat['scores'][0]['postId']) for stat in stats],
    #             y = [len(stat['vocabularyList']) for stat in stats],
    #             name = 'Amount of saved words')
                data.append(trace1)
    layout = go.Layout(
        title = "Label:" +str(label_no)
    )
    fig = go.Figure(data=data[-20:-1], layout=layout)
    po.iplot(fig)

In [ ]:
for student in student_summary:
    if student['memberId'] == '10026':
        for stat in student['stats']:
            if stat['postId'] == 4220:
                print stat['vocabularyList']